In [1]:
from flask import Blueprint
import numpy as np
import pandas as pd
import pickle
import copy
from sklearn.preprocessing import MinMaxScaler
import requests
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
from pandas import Series
from numpy import nan
import copy
import math

In [55]:
from glob import glob
import os
import pandas as pd
# from flask import jsonify/

path = os.getcwd()
csv_list = sorted(glob(os.path.join(path,"DB","*.csv")))

data = pd.read_csv(csv_list[1])
data.iloc[0,0]

5.1

In [8]:
from flask import Blueprint
from glob import glob
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def train(model, x, y):
  with tf.GradientTape() as t:
    # Trainable variables are automatically tracked by GradientTape
    current_loss = loss(y_true=y,y_pred=model(x, training=True))

  # Use GradientTape to calculate the gradients with respect to W and b
  grads= t.gradient(current_loss, model.trainable_variables)
  
  # Subtract the gradient scaled by the learning rate
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

def training_loop(model, x, y, validset = None):
    # Keep results for plotting
    train_loss_results = []
    val_loss_results = []
    for epoch in epochs:
        # Update the model with the single giant batch
        train(model, x, y)

        # Track this before I update
        if validset == None:
            current_loss = loss(y,model(x))
            print("Epoch %2d: loss=%2.5f" %
                (epoch, current_loss))
        else:
            x_val = validset[0]
            y_val = validset[1]

            current_loss = loss(y_true=y,y_pred=model(x))
            valid_loss = loss(y_true=y_val,y_pred=model(x_val))
            print("Epoch %2d: loss=%2.5f  val_loss=%2.5f" %
                (epoch, current_loss, valid_loss))
            # save loss history
            train_loss_results.append(current_loss.numpy())
            val_loss_results.append(valid_loss.numpy())
    return train_loss_results,val_loss_results
            


path = os.getcwd()
csv_list = sorted(glob(os.path.join(path,"DB","*.csv")))
data = pd.read_csv(csv_list[1])
trainset, testset = train_test_split(data,test_size=0.2,shuffle=True)

scaler = MinMaxScaler()
X_train = trainset.values[:,:-1]
X_train = scaler.fit_transform(X_train)
y_train = trainset.values[:,-1]
# y_train = pd.get_dummies(trainset.values[:,-1]).values
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)

X_test = testset.values[:,:-1]
X_test = scaler.transform(X_test)
y_test = testset.values[:,-1]
# y_test = pd.get_dummies(testset.values[:,-1]).values
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)


#params
input_shape = 4
output_shape = 3
lr = 0.03
act_function = 'relu'
outlayer_act_function = 'softmax'
category = "classification"
#optimizer
optimizer = keras.optimizers.SGD(learning_rate=lr)

# #loss function
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


num_perceptron_layer1 = 8
num_perceptron_layer2 = 16
num_perceptron_layer3 = 32
num_perceptron_layer4 = 16
num_perceptron_layer5 = 8
num_perceptron_out = output_shape

num_perceptron_list = [
    num_perceptron_layer1,
    num_perceptron_layer2,
    num_perceptron_layer3,
    num_perceptron_layer4,
    num_perceptron_layer5,
    num_perceptron_out,
    ]

# model definition
inputs = tf.keras.Input(shape=input_shape)
for i in range(len(num_perceptron_list)):
    n_percep = num_perceptron_list[i]
    if i == 0:
        x = keras.layers.Dense(n_percep, activation= act_function)(inputs)

    elif  i != 0 and n_percep != 0 and i != len(num_perceptron_list)-1:
        x = keras.layers.Dense(n_percep, activation= act_function)(x)

    elif  i == len(num_perceptron_list)-1:
        x = keras.layers.Dense(n_percep,)(x)#activation= outlayer_act_function)(x)

    else: pass

dense_model = tf.keras.Model(name="model_1",inputs=inputs, outputs=x)
# dense_model.summary()


# Define a training loop
epochs = range(300)    
# print(y_train[:5],dense_model(X_train[:5]))
# print(loss(y_train[:5],dense_model(X_train[:5])))
# print(loss(y_train[:5], dense_model(X_train[:5])))
# Do the training
# training_loop(dense_model, X_train, y_train,validset=[X_test,y_test])


# from sklearn import metrics
# y_pred_test = dense_model(X_test).numpy()
# y_pred_test = np.argmax(y_pred_test, axis=1).tolist()
# # y_test = np.argmax(y_test, axis=1).tolist()

# # Print the confusion matrix
# print(metrics.confusion_matrix(y_test, y_pred_test))


In [9]:
train_loss, val_loss = training_loop(dense_model, X_train, y_train,validset=[X_test,y_test])

Epoch  0: loss=1.07314  val_loss=1.09133
Epoch  1: loss=1.07192  val_loss=1.09090
Epoch  2: loss=1.07084  val_loss=1.09046
Epoch  3: loss=1.06986  val_loss=1.08993
Epoch  4: loss=1.06888  val_loss=1.08938
Epoch  5: loss=1.06790  val_loss=1.08879
Epoch  6: loss=1.06691  val_loss=1.08818
Epoch  7: loss=1.06593  val_loss=1.08757
Epoch  8: loss=1.06494  val_loss=1.08696
Epoch  9: loss=1.06394  val_loss=1.08633
Epoch 10: loss=1.06291  val_loss=1.08569
Epoch 11: loss=1.06187  val_loss=1.08502
Epoch 12: loss=1.06081  val_loss=1.08433
Epoch 13: loss=1.05973  val_loss=1.08363
Epoch 14: loss=1.05862  val_loss=1.08292
Epoch 15: loss=1.05749  val_loss=1.08219
Epoch 16: loss=1.05635  val_loss=1.08144
Epoch 17: loss=1.05518  val_loss=1.08065
Epoch 18: loss=1.05400  val_loss=1.07983
Epoch 19: loss=1.05281  val_loss=1.07899
Epoch 20: loss=1.05159  val_loss=1.07812
Epoch 21: loss=1.05038  val_loss=1.07725
Epoch 22: loss=1.04918  val_loss=1.07645
Epoch 23: loss=1.04797  val_loss=1.07566
Epoch 24: loss=1

In [10]:
train_loss

[1.0731417,
 1.0719198,
 1.0708374,
 1.069859,
 1.0688819,
 1.0678983,
 1.0669118,
 1.0659312,
 1.0649389,
 1.0639391,
 1.062912,
 1.061874,
 1.0608132,
 1.0597281,
 1.0586226,
 1.0574933,
 1.056346,
 1.0551758,
 1.054001,
 1.0528057,
 1.0515925,
 1.0503774,
 1.049179,
 1.0479696,
 1.0467415,
 1.0455058,
 1.0442542,
 1.0429847,
 1.0417013,
 1.0403972,
 1.0390639,
 1.0376979,
 1.0363002,
 1.0348952,
 1.0334691,
 1.0320154,
 1.0305359,
 1.0290105,
 1.0274485,
 1.025848,
 1.0242429,
 1.0226077,
 1.0209503,
 1.0192816,
 1.0175245,
 1.0156823,
 1.0135654,
 1.0110853,
 1.0082625,
 1.0054735,
 1.0028225,
 1.0002545,
 0.997798,
 0.99546564,
 0.9931856,
 0.99093145,
 0.9886743,
 0.98640287,
 0.98412627,
 0.9818436,
 0.9795442,
 0.97723424,
 0.97490275,
 0.9725569,
 0.97019756,
 0.967823,
 0.9654325,
 0.9630274,
 0.9606079,
 0.9581722,
 0.95572525,
 0.95326,
 0.95077515,
 0.94826764,
 0.94568425,
 0.94281924,
 0.9398394,
 0.93723094,
 0.9346712,
 0.9321108,
 0.929548,
 0.92696005,
 0.92436004,
 